In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TwitterAnalysis").getOrCreate()

# Load JSON data into a DataFrame
twitter_df = spark.read.json("file:///home/hduser/Desktop/Trump_Tweets.json")

2023-05-18 14:34:05,147 WARN util.Utils: Your hostname, muhammad-Vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2023-05-18 14:34:05,156 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
2023-05-18 14:34:07,321 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
twitter_df.show()

+-------------------+-------------------+----------+-----------+--------------------+
|             Client|  Client Simplified|      Date|       Time|               Tweet|
+-------------------+-------------------+----------+-----------+--------------------+
| Twitter for iPhone| Twitter for iPhone|01/27/2017| 6:46:22 PM| I promise that o...|
| Twitter for iPhone| Twitter for iPhone|01/27/2017| 5:00:47 PM| Congratulations ...|
| Twitter for iPhone| Twitter for iPhone|01/27/2017| 3:20:15 PM| Statement on Int...|
| Twitter for iPhone| Twitter for iPhone|01/27/2017|11:30:29 AM| .@VP Mike Pence ...|
| Twitter for iPhone| Twitter for iPhone|01/27/2017|11:27:02 AM| The #MarchForLif...|
|Twitter for Android|Twitter for Android|01/27/2017| 8:19:10 AM| Mexico has taken...|
|Twitter for Android|Twitter for Android|01/27/2017| 8:12:52 AM| Look forward to ...|
| Twitter for iPhone| Twitter for iPhone|01/26/2017| 6:53:37 PM| Miami-Dade Mayor...|
| Twitter for iPhone| Twitter for iPhone|01/26/2017| 6